In [1]:
%reload_ext autoreload
%autoreload 2

! [ -L /datasets ] && rm -f /datasets
! ln -s /data/datasets/cv /datasets

from k12libs.utils.nb_easy import k12ai_set_notebook

k12ai_set_notebook(cellw=95)

## 需掌握知识点

1. 不确定性与概率
2. 准确率Top分级
3. 置信度和混淆矩阵

In [108]:
%reset -f

from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer
import torch
from torch import nn
from sklearn.metrics import confusion_matrix

class CustomClassifier(EasyaiClassifier):
    
    def __init__(self):
        super().__init__()
        self.input_size = 100 # 输入模型的图片大小
        self.batch_size = 32  # 输入模型的图片数量
        
        # 调试: 日志输出模型shape
        self.example_input_array = torch.zeros(
            self.batch_size, 3, self.input_size, self.input_size) 

    def prepare_dataset(self):
        """
        水果数据集:
            0: 'Banana'  香蕉
            1: 'Pear'    梨子
            2: 'Lemon'   柠檬
            3: 'Apple'   苹果
            4: 'Orange'  橘子
        """
        return self.load_fruits()
    
    def build_model(self):
        class ConvNetwork(nn.Module):
            def __init__(self, num_classes):
                super().__init__()
                self.features = nn.Sequential(
                    nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1, padding=1, dilation=1, groups=1, bias=True),
                    nn.ReLU(inplace=True),
                    nn.AvgPool2d(kernel_size=5, stride=2, padding=1, ceil_mode=False),
                    nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True),
                    nn.BatchNorm2d(num_features=16, momentum=0.1, affine=True, track_running_stats=True),
                    nn.ReLU(inplace=True),
                    nn.AvgPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=False)
                )
                self.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(in_features=36864, out_features=5, bias=True)
                )
                
            def forward(self, x):
                x = self.features(x)
                return self.classifier(x)
            
        return ConvNetwork(num_classes=5) # input shape: (32, 3, 100, 100)
    
    def train_dataloader(self):
        return self.get_dataloader('train', self.batch_size, self.input_size)
     
    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)
        loss = self.cross_entropy(y_hat, y, reduction='mean') # 损失方法
        with torch.no_grad():
            accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        return {'loss': loss, 'progress_bar': {'acc': accuracy}}

    def val_dataloader(self):
        return self.get_dataloader('val', self.batch_size, self.input_size)
    
    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)
        loss = self.cross_entropy(y_hat, y, reduction='mean')
        accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        return {'loss': loss, 'acc': accuracy}
        
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        return {'progress_bar': {'val_loss': avg_loss, 'val_acc': avg_acc}}
    
    def test_dataloader(self):
        return self.get_dataloader('test', self.batch_size, self.input_size)
    
    def test_step(self, batch, batch_idx):
        x, y, _ = batch  # y.size: (B,)
        y_hat = self(x)
        return {'y_true': y, 'y_pred': y_hat}
    
    def test_epoch_end(self, outputs):
        y_true = torch.cat([x['y_true'].cpu() for x in outputs])
        y_pred = torch.cat([x['y_pred'].cpu() for x in outputs])
        
        # Top分级
        _, y_top5 = y_pred.topk(5, 1)
        print(y_top5[:, 0])

        corrects = y_top5.eq(y_true.view(-1, 1).expand_as(y_top5))
        acc_top1 = corrects[:, :1].float().sum() / y_top5.size()[0]
        acc_top3 = corrects[:, :3].float().sum() / y_top5.size()[0]
        acc_top5 = corrects[:, :5].float().sum() / y_top5.size()[0]
        
        # 混淆矩阵
        cm = confusion_matrix(y_true, y_top5[:, 0])
        return [{'acc_top1': acc_top1, 'acc_top3': acc_top3, 'acc_top5': acc_top5}, {'cm': cm}]
        
    
    def configure_optimizer(self, model):
        return self.adam(model.parameters(), base_lr=0.1)
    
    
trainer = EasyaiTrainer(max_epochs=1, model_summary='full', ckpt_path='5-1')

model = CustomClassifier()

# 训练
trainer.fit(model)

# 评估
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
{'label_names': ['Banana', 'Pear', 'Lemon', 'Apple', 'Orange'],
 'mean': [0.7507, 0.6427, 0.5061],
 'num_classes': 5,
 'num_records': 3263,
 'std': [0.2251, 0.2804, 0.3714]}

--------------------------------------------------------------------------------

   | Name               | Type        | Params | In sizes          | Out sizes       
-------------------------------------------------------------------------------------------
0  | model              | ConvNetwork | 186 K  | [32, 3, 100, 100] | [32, 5]         
1  | model.features     | Sequential  | 1 K    | [32, 3, 100, 100] | [32, 16, 48, 48]
2  | model.features.0   | Conv2d      | 608    | [32, 3, 100, 100] | [32, 8, 98, 98] 
3  | model.features.1   | ReLU        | 0      | [32, 8, 98, 98]   | [32, 8, 98, 98] 
4  | model.features.2   | AvgPool2d   | 0      | [32, 8, 98, 98]   | [32, 8, 48, 48] 
5  | model.features.3   | Conv2d      | 1 K    | [32, 

Saving latest checkpoint..



(fit)	GPU-0 memory allocated: 2.18 MB	 max memory allocated: 41.23 MB
--------------------------------------------------------------------------------
{'label_names': ['Banana', 'Pear', 'Lemon', 'Apple', 'Orange'],
 'mean': [0.7507, 0.6427, 0.5061],
 'num_classes': 5,
 'num_records': 3263,
 'std': [0.2251, 0.2804, 0.3714]}


tensor([3, 2, 1, 0, 0, 4, 4, 1, 4, 0, 1, 4, 1, 3, 3, 0, 0, 4, 1, 4, 4, 3, 0, 0,
        3, 1, 2, 0, 0, 3, 3, 2, 4, 0, 0, 3, 2, 4, 3, 4, 4, 1, 1, 4, 0, 4, 2, 0,
        0, 3, 2, 0, 3, 3, 1, 4, 1, 0, 1, 2, 4, 3, 2, 4, 3, 0, 1, 2, 0, 4, 2, 4,
        4, 0, 2, 4, 1, 3, 4, 4, 0, 4, 1, 0, 4, 3, 4, 0, 3, 3, 2, 3, 0, 1, 1, 0,
        3, 3, 4, 3, 2, 3, 4, 4, 3, 4, 0, 2, 3, 4, 2, 3, 1, 0, 0, 0, 0, 0, 4, 3,
        1, 0, 2, 4, 1, 1, 2, 2, 2, 4, 0, 2, 0, 4, 2, 2, 1, 2, 3, 4, 0, 4, 3, 4,
        1, 4, 4, 3, 2, 3, 1, 4, 1, 0, 2, 4, 4, 1, 0, 2, 4, 2, 1, 3, 3, 4, 4, 2,
        4, 4, 4, 2, 0, 3, 1, 4, 1, 4, 4, 3, 1, 2, 4, 2, 2, 4, 1, 2, 2, 2, 4, 3,
        4, 4, 0, 0, 4, 1, 1, 3, 1, 3, 1, 1, 4, 2, 1, 1, 3, 0, 0, 0, 3, 1, 1, 2,
        3, 4, 4, 0, 4, 4, 0, 2, 1, 2, 4, 1, 4, 2, 4, 4, 4, 4, 1, 0, 0, 3, 0, 1,
        4, 0, 2, 1, 4, 1, 1, 4, 4, 1, 2, 4, 3, 0, 3, 1, 4, 3, 0, 0, 0, 1, 3, 0,
        0, 4, 4, 2, 1, 3, 3, 0, 4, 0, 0, 1, 4, 1, 1, 0, 2, 4, 2, 4, 4, 3, 1, 1,
        1, 1, 1, 1, 0, 1, 2, 3, 4, 0, 1,